In [80]:
from importnb import Notebook, reload
from nbformat import v4
from pathlib import Path
import shutil, os, functools
from pytest import fixture, mark


In [73]:
def new_notebook(str='foo') -> str:
    """Stringify a new notebook to test with a simple set of instructions that may be formatter.
    
    >>> assert isinstance(new_notebook(), str)
    """
    return v4.writes(v4.new_notebook(cells=[
            v4.new_code_cell("""foo = 42\nassert {}\nbar= 100""".format(str))]))

# Test Single File Modules

Single file modules mimic common Untitled notebooks.  An author should be able to trivially import notebooks in their working directory.

## Single File Fixtures

In [75]:
@fixture(scope='function')
def single_file(request):
    """A fixture to write a new notebook to disk and delete after each function call."""
    file = Path('foobar.ipynb')
    file.write_text(new_notebook())
    request.addfinalizer(functools.partial(os.remove, file))
    return file

Each time a file is imported we should clear up the sys path to reset our imports and assure the validity of our tests.

In [76]:
@fixture
def clean_up_file(single_file, request):
    def clean_sys():
        import sys
        del sys.modules['foobar']
        sys.path_importer_cache.clear()
    request.addfinalizer(clean_sys)

`importnb`'s most generic use is as a context manager.  `with Notebook()` will update the `sys.path_hooks` to import notebooks as modules.

In [57]:
def test_single_file_with_context(clean_up_file):
    with Notebook():
        import foobar
    assert foobar.foo == 42 and foobar.bar == 100
    
    validate_reload(foobar)

Each time we test a notebook import we should test the ability to reload the module.  `importnb` expresses the ability to use the normal Python import system, and a notebook must reload for interactive development.

In [77]:
def validate_reload(module):
    try:
        reload(module)
        assert False, """The reload should have fail."""
    except:
        assert True, """Cannot reload a file outside of a context manager"""

    with Notebook():
        assert reload(module)

A notebook will not import without the context manager or [IPython extension](#IPython-extension).

In [ ]:
@mark.xfail
def test_single_file_without_context():
    import foobar

In the `__main__` context, relative imports are not allowed. 

In [78]:
@mark.xfail
def test_single_file_relative(single_file):
    with Notebook():
        from . import foobar

Commonly, we use the `try` statement to allow the ability to use relative imports in a package while developing interactively.

    try:
        from . import a_module
    except:
        import a_module

## IPython extension

In general, an author would use IPython sugar to load an extension

    %load_ext importnb
    
For testing purposes we use the explicit functions to create the extensions

In [83]:
from importnb import load_ipython_extension, unload_ipython_extension

In [84]:
@fixture
def extension(clean_up_file, request):
    load_ipython_extension()
    request.addfinalizer(unload_ipython_extension)

In [85]:
def test_single_with_extension(extension):
    import foobar
    assert foobar.foo == 42 and foobar.bar == 100

In [86]:
@fixture
def single_directory(request):
    root = Path('a_test_package')
    root.mkdir(exist_ok=True)
    (root / 'foobar.ipynb').write_text(new_notebook())
    (root / 'failure.ipynb').write_text(new_notebook('False'))
    (root / 'py.py').write_text("""from . import foobar\nbaz = 'foobar'""")
    request.addfinalizer(functools.partial(shutil.rmtree, root))
    return root

In [87]:
def test_package(single_directory):
    with Notebook():
        from a_test_package import foobar, py
        
    assert foobar.foo == 42 and foobar.bar == 100
    assert py.baz == 'foobar'
    assert py.foobar is foobar
    validate_reload(foobar)

In [88]:
@mark.xfail
def test_package_failure(single_directory):
    with Notebook():
        from a_test_package import failure

## Partial Imports.

In [90]:
from importnb import Partial

In [91]:
def test_package_failure_partial(single_directory):
    with Partial():
        from a_test_package import failure
        
    assert isinstance(failure.__exception__, AssertionError), """
    The wrong error was returned likely because of importnb."""

    from traceback import print_tb
    from io import StringIO
    s = StringIO()
    print_tb(failure.__exception__.__traceback__, file=s)
    print(s.getvalue())
    assert """a_test_package/failure.ipynb", line 10, in <module>\n""" in s.getvalue(), """Traceback is not satisfied"""